<a href="https://colab.research.google.com/github/hananbahtiti/Hybrid-Intrusion-detection-Systems/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#test

In [4]:
!wget -O NGIDS-DS.rar https://unsworks.unsw.edu.au/bitstreams/0ac2820a-5131-43ab-90b2-c624c8d73649/download

--2025-04-19 18:48:59--  https://unsworks.unsw.edu.au/bitstreams/0ac2820a-5131-43ab-90b2-c624c8d73649/download
Resolving unsworks.unsw.edu.au (unsworks.unsw.edu.au)... 54.253.215.118
Connecting to unsworks.unsw.edu.au (unsworks.unsw.edu.au)|54.253.215.118|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://unsworks.unsw.edu.au/server/api/core/bitstreams/0ac2820a-5131-43ab-90b2-c624c8d73649/content [following]
--2025-04-19 18:49:01--  https://unsworks.unsw.edu.au/server/api/core/bitstreams/0ac2820a-5131-43ab-90b2-c624c8d73649/content
Reusing existing connection to unsworks.unsw.edu.au:443.
HTTP request sent, awaiting response... 200 200
Length: unspecified [application/x-rar-compressed]
Saving to: ‘NGIDS-DS.rar’

NGIDS-DS.rar            [         <=>        ] 941.51M  15.7MB/s    in 61s     

2025-04-19 18:50:02 (15.5 MB/s) - ‘NGIDS-DS.rar’ saved [987249484]



In [5]:
!unrar x /content/NGIDS-DS.rar


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/NGIDS-DS.rar

Creating    NGIDS                                                     OK
Creating    NGIDS/NGIDS-DS-v1                                         OK
Extracting  NGIDS/NGIDS-DS-v1/feature_descr.csv                            0%  OK 
Extracting  NGIDS/NGIDS-DS-v1/ground_truth.csv                             0%  OK 
Creating    NGIDS/NGIDS-DS-v1/host logs                               OK
Extracting  NGIDS/NGIDS-DS-v1/host logs/1.csv                              0%  OK 
Extracting  NGIDS/NGIDS-DS-v1/host logs/10.csv                             0%  1%  OK 
Extracting  NGIDS/NGIDS-DS-v1/host logs/11.csv                             1%  OK 
Extracting  NGIDS/NGIDS-DS-v1/host logs/12.csv                             1%  2%  OK 
Extracting  NGIDS/NGIDS-DS-v1/host logs/13.csv                             2%  OK 
Extrac

#class

In [6]:
import pandas as pd
import numpy as np
import os
import glob
import math
from sklearn.preprocessing import LabelEncoder

In [ ]:
class Preprocessing():
  def __init__(self):
    pass

  def __file_path_collection(self, folder_path):
    try:
      joined_list = []
      for file_name in os.listdir(folder_path):
        file_name = os.path.join(folder_path, file_name)
        if os.path.isfile(file_name) and file_name.endswith('.csv'):
          joined_list.append(file_name)
      return joined_list

    except Exception as e:
      return f"An error occurred: {e}"


  def rename_columns(self, folder_path):
    files = self.__file_path_collection(folder_path)
    try:
      for file in files:
        df = pd.read_csv(file)
        column_name = df.columns.tolist()
        df.rename(columns={column_name[0]:'Date', column_name[1]:'Time', column_name[2]:'Unique_Identification', column_name[3]:'Execution_Path', column_name[4]:'System_Calls_Identifiers', column_name[5]:"Event's_Unique_Identification", column_name[6]:'attacks', column_name[7]:"Sub_Type_Attack", column_name[8]:"Label"}, inplace=True)
        df.to_csv(file, index=False)
      return f"done files..."

    except Exception as e:
      return f"An error occurred: {e}"


  def data_encoding(self, folder_path):
    files = self.__file_path_collection(folder_path)
    encoder = LabelEncoder()
    all_value = []
    C=0
    try:
      for file in files:
        df = pd.read_csv(file)
        all_value.extend(df['attacks'].dropna().unique())
        print(C)
        C=C+1
      encoder.fit(all_value)

      for file in files:
        df = pd.read_csv(file)
        print(file, 1)
        df['attacks'] = encoder.transform(df['attacks'])
        df.to_csv(file, index=False)
      return f"done file..."

    except Exception as e:
      return f"An error occurred: {e}"



  def file_collection(self, folder_path):
    try:
      file_paths = self.__file_path_collection(folder_path)
      if file_paths:
        output_folder = os.path.join(os.getcwd(),'files')
        os.makedirs(output_folder ,exist_ok=True)
        output_file = os.path.join(output_folder,"final.csv")

        for file in file_paths:

          first = True
          chunk = pd.read_csv(file, chunksize=10000)
          for part in chunk:
            part.to_csv(output_file, mode='a', header=first, index=False)
            first = False
          print(f"done file: {file}")

    except Exception as e:
      print(f"An error occurred: {e}")


a = Preprocessing()
#a.file_path_collection("/content/NGIDS/NGIDS-DS-v1/host logs")
#a.file_collection("/content/NGIDS/NGIDS-DS-v1/host logs")
#a.rename_columns( '/content/NGIDS/NGIDS-DS-v1/host logs')
a.data_encoding('/content/NGIDS/NGIDS-DS-v1/host logs')

0
1
2
3
4
5
6
7
8
9


In [ ]:
unique_attacks = list(set([item for sub in c for item in sub]))
unique_attacks

['Exploits',
 'Generic',
 'Shellcode',
 'Worms',
 'Reconnaissance',
 'Denial of Service',
 'Backdoors',
 'normal']

In [ ]:
def data_encoding(self, folder_path):
    files = self.__file_path_collection(folder_path)
    encoder = LabelEncoder()
    max_count = 0
    max_file = ""
    names_attacks = []
    try:
      for file in files:
        df = pd.read_csv(file)
        names_attacks.append([df['attacks'].unique(), len(df['attacks'].unique()),file])

      for names_attack, count, file in names_attacks:
        if count > max_count:
          max_count = count
          max_file = file
          max_names_attacks = names_attack
      return max_names_attacks, max_count, max_file
    except Exception as e:
      return f"An error occurred: {e}"

'normal'

In [14]:
pd.read_csv('/content/NGIDS/NGIDS-DS-v1/host logs/1.csv')

,Date,Time,Unique_Identification,Execution_Path,System_Calls_Identifiers,Event's_Unique_Identification,attacks,Sub_Type_Attack,Label
0,11/03/2016,2:45:06,1804,/bin/dbus-daemon,256,45352,normal,normal,0
1,11/03/2016,2:45:06,2133,/usr/lib/i386-linux-gnu/gconf/gconfd-2,168,45372,normal,normal,0
2,11/03/2016,2:45:35,4528,/usr/bin/python3.4,3,39459,normal,normal,0
3,11/03/2016,2:45:44,1847,/usr/bin/ibus-daemon,102,37263,normal,normal,0
4,11/03/2016,2:45:44,1907,/usr/lib/ibus/ibus-ui-gtk3,168,37896,normal,normal,0
...,...,...,...,...,...,...,...,...,...
999995,11/03/2016,4:09:37,2110,/usr/bin/compiz,265,1426117,normal,normal,0
999996,11/03/2016,4:04:53,2110,/usr/bin/compiz,102,1349650,normal,normal,0
999997,11/03/2016,4:04:18,2110,/usr/bin/compiz,265,1337834,normal,normal,0
999998,11/03/2016,4:04:53,2110,/usr/bin/compiz,102,1349753,normal,normal,0
